<p style="font-family: Arial; font-size:3.75em;color:purple; font-style:bold"><br>
'SiPM DST' Study</p><br>

# Prepares an analysis iber study from FLEX data

In [1]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [2]:
# General importings
import os
import sys
import glob
import logging
import math
import numpy  as np
import warnings
from scipy.stats import norm
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors

# Specific IC stuff
import invisible_cities.core.system_of_units  as units

from invisible_cities.io.mcinfo_io import load_mcconfiguration
from invisible_cities.io.mcinfo_io import load_mcparticles_df
from invisible_cities.io.mcinfo_io import load_mchits_df
from invisible_cities.io.mcinfo_io import load_mcsensor_positions
from invisible_cities.io.mcinfo_io import load_mcsensor_response_df
from invisible_cities.io.mcinfo_io import get_sensor_types
from invisible_cities.io.mcinfo_io import get_sensor_binning
from invisible_cities.io.mcinfo_io import get_event_numbers_in_file
from invisible_cities.core.core_functions import in_range


In [3]:
from pynext.pynext_types import SiPM

In [4]:
from nextflex.krypton_dst import get_evt_true_positions_and_energy

In [5]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [6]:
plt.rcParams["figure.figsize"] = 8, 6
plt.rcParams["font.size"     ] = 14

# Code

# DATA

In [7]:
VERBOSITY = False

# Input data

In [8]:
iPATH = "/Users/jj/Development/demoData/flexkr1ns"
file ='NEXT_FLEX.Kr83.ACTIVE.0.next.h5'
path =f'{iPATH}/{file}'

In [9]:
ifnames = glob.glob(f"{iPATH}/NEXT_FLEX.Kr83.ACTIVE*.h5")

if VERBOSITY:
    print(f"{len(ifnames)} input file names ...\n")
    for ifname in ifnames:
        print(ifname)
        #print(get_event_numbers_in_file(ifname))
else:
    print(f"{len(ifnames)} input file names ...\n")

249 input file names ...



In [10]:
# Verbosing main config parameters
mcConfig = load_mcconfiguration(ifnames[0])
mcConfig.set_index("param_key", inplace = True)
main_params = ["/Geometry/NextFlex/e_lifetime", "/Geometry/NextFlex/el_gap_length",
               "/Geometry/NextFlex/el_field_int", "/Geometry/NextFlex/tp_sipm_anode_dist",
               "/Geometry/NextFlex/tp_teflon_thickness", "/Geometry/NextFlex/tp_teflon_hole_diam",
               "/Geometry/NextFlex/tp_sipm_sizeX", "/Geometry/NextFlex/tp_sipm_pitchX",
               "num_events", "TP_SiPM_binning", "F_SENSOR_L_binning", "F_SENSOR_R_binning"]
print(mcConfig.loc[main_params])


                                                param_value
param_key                                                  
/Geometry/NextFlex/e_lifetime                     10000. ms
/Geometry/NextFlex/el_gap_length                    10.  mm
/Geometry/NextFlex/el_field_int             16. kilovolt/cm
/Geometry/NextFlex/tp_sipm_anode_dist              13.1  mm
/Geometry/NextFlex/tp_teflon_thickness               2.1 mm
/Geometry/NextFlex/tp_teflon_hole_diam               3.  mm
/Geometry/NextFlex/tp_sipm_sizeX                    1.0  mm
/Geometry/NextFlex/tp_sipm_pitchX                  15.55 mm
num_events                                              200
TP_SiPM_binning                                1.000000 mus
F_SENSOR_L_binning                             0.001000 mus
F_SENSOR_R_binning                             0.001000 mus


In [11]:
par = mcConfig.loc["/Geometry/NextFlex/tp_sipm_pitchX"]
par

param_value           15.55 mm
Name: /Geometry/NextFlex/tp_sipm_pitchX, dtype: object

## Extract pitch from params

In [12]:
pitch = float(par.param_value.split(' ')[-2])
pitch

15.55

### SiPMs are at indices < 100000 in dataframes 

In [13]:
sipm_index = 100000

## read SiPM map

In [14]:
sipm_map = pd.read_csv('sipm_map.csv')

In [15]:
sipm_map

,Unnamed: 0,sensor_id,sensor_name,x,y,xl,xr,yu,yd,id_xl,id_xr,id_yu,id_yd
0,0,1040,TP_SiPM,15.55,-450.95,1.907349e-07,31.100000,-435.400012,-466.500012,1039,1041,1066,1019
1,1,4059,TP_SiPM,93.30,450.95,7.775000e+01,108.850003,466.500012,435.400012,4058,4060,4080,4033
2,2,3753,TP_SiPM,-62.20,326.55,-7.775000e+01,-46.650001,342.099988,310.999988,3752,3754,3799,3705
3,3,3842,TP_SiPM,-77.75,357.65,-9.330000e+01,-62.200000,373.199994,342.099994,3841,3843,3884,3798
4,4,3797,TP_SiPM,-93.30,342.10,-1.088500e+02,-77.750003,357.650006,326.550006,3796,3798,3841,3751
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3088,3088,1717,TP_SiPM,311.00,-217.70,2.954500e+02,326.550000,-202.149997,-233.249997,1716,1718,1774,1661
3089,3089,2317,TP_SiPM,357.65,-62.20,3.421000e+02,373.199994,-46.650001,-77.750001,2316,2318,2380,2255
3090,3090,1676,TP_SiPM,-326.55,-217.70,-3.421000e+02,-310.999988,-202.149997,-233.249997,1675,1677,1733,1620
3091,3091,3517,TP_SiPM,186.60,248.80,1.710500e+02,202.150006,264.350003,233.250003,3516,3518,3570,3463


## Sensors

In [16]:
get_sensor_binning(path)

,bin_width
sns_name,
F_SENSOR_L,1.0
F_SENSOR_R,1.0
TP_SiPM,1000.0


In [17]:
sns_types = get_sensor_types(path)
sns_types.head()

,sensor_id,sensor_name
0,100015,F_SENSOR_L
1,101428,F_SENSOR_L
2,100241,F_SENSOR_L
3,100284,F_SENSOR_L
4,100215,F_SENSOR_L


In [18]:
sns_positions = load_mcsensor_positions(path)
sns_positions.head()

,sensor_id,sensor_name,x,y,z
0,100015,F_SENSOR_L,30.055239,492.083008,-30.299999
1,101428,F_SENSOR_L,-225.824677,438.237640,-30.299999
2,100241,F_SENSOR_L,409.461456,274.572998,-30.299999
3,100284,F_SENSOR_L,450.987427,199.146545,-30.299999
4,100215,F_SENSOR_L,378.196350,316.253906,-30.299999


## SENSORS RESPONSE

In [19]:
sns_response = load_mcsensor_response_df(ifnames[0])
sns_response

charge       time
event_id sensor_id                   
43800    100130          1   809404.0
         100130          1   812659.0
         100130          1   813046.0
         100130          1   813649.0
         100130          1   813677.0
...                    ...        ...
43983    3305            1   603000.0
43984    3305            1   365000.0
         3305            1   369000.0
43985    3305            1  1129000.0
43991    3305            1   841000.0

[11093268 rows x 2 columns]

In [20]:
sipm_response = sns_response[sns_response.index \
                               .get_level_values("sensor_id") < sipm_index]

In [21]:
sipm_response

charge       time
event_id sensor_id                   
43800    3707            2   811000.0
         3707            3   812000.0
         3707            8   813000.0
         3707            4   814000.0
         3707           10   815000.0
...                    ...        ...
43983    3305            1   603000.0
43984    3305            1   365000.0
         3305            1   369000.0
43985    3305            1  1129000.0
43991    3305            1   841000.0

[187608 rows x 2 columns]

In [22]:
fibers_response = sns_response[sns_response.index.get_level_values("sensor_id") >= sipm_index]

In [23]:
fibers_response

charge      time
event_id sensor_id                  
43800    100130          1  809404.0
         100130          1  812659.0
         100130          1  813046.0
         100130          1  813649.0
         100130          1  813677.0
...                    ...       ...
43999    201328          1  157726.0
         201328          1  157875.0
         201328          1  157882.0
         201328          1  158916.0
         201328          1  159070.0

[10905660 rows x 2 columns]

In [27]:
fevt0 = fibers_response[fibers_response.index.get_level_values("event_id") == 43800]

In [28]:
fevt0

charge      time
event_id sensor_id                  
43800    100130          1  809404.0
         100130          1  812659.0
         100130          1  813046.0
         100130          1  813649.0
         100130          1  813677.0
...                    ...       ...
         201328          1  814117.0
         201328          1  814363.0
         201328          1  814486.0
         201328          1  815430.0
         201328          1  819296.0

[52628 rows x 2 columns]